In [ ]:
#!pip3 install selenium
#!pip3 install tqdm
#!pip3 install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

In [7]:
import pandas as pd
from datetime import datetime
from datetime import date 

from time import sleep
from tqdm.notebook import tqdm, trange

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

## Bitso Historical Live Scraping

In [8]:
coins=['btc','eth','xrp','ltc','bch','tusd','mana','gnt','bat','dai']
URLs={}
for i in coins:
    URLs.update({i:'https://bitso.com/trade/market/'+i+'/mxn'})

In [9]:
def readscreen(test=False, verbose=False, database=[]):
    try:
        # Getting candels tags
        tagsPath = driver.find_elements_by_tag_name('g')
        candels=[]
        for i in range(len(tagsPath)):
            if tagsPath[i].get_attribute("aria-label") != None and tagsPath[i].get_attribute("aria-label").find('Price')==0:
                #print(tagsPath[i].get_attribute("transform"), tagsPath[i].get_attribute("aria-label"))
                candels.append(tagsPath[i])
        print("\n",len(candels), 'Candels detected')

        #Checking every candel by moving arround
        
        for i in trange(len(candels)):
            try:
                action=ActionChains(driver)
                action.move_to_element(candels[i])
                action.perform()
                #print(i, candels[i].get_attribute('aria-label'))
                
                #Getting info of every candel

                voltag=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div/div/div')
                volume=voltag.text

                boxtags = driver.find_elements_by_tag_name('b')
                i_open=boxtags[0].text
                i_low=boxtags[1].text
                i_high=boxtags[2].text
                i_close=boxtags[3].text

                datetag=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[1]/div/div/div[2]/div[1]/div/div/div[2]/div')
                date_str=datetag.text
                i_date=datetime.strptime(date_str,'%b %d, %Y')

                day=[i_date,i_open,i_low,i_high,i_close,volume] #Register integration
                
                if test == False:
                    database.append(day)

                if verbose == True:
                    print(day)
            except:
                print('The Candel',i,'Fails when was tried to read it')
        return day[0].date() #Control date to finish the scanning of windows dates
    except:
            print('ERROR: Mayor Fail - Check the driver is initialized correctly')
   

In [10]:
def getScrollerPosition(si,sf):
    xi,yi=int(si[si.find('(')+1:si.find(',')]),int(si[si.find(',')+1:si.find(')')])
    xf,yf=int(sf[sf.find('(')+1:sf.find(',')]),int(sf[sf.find(',')+1:sf.find(')')])
    return xi,yi,xf,yf

In [11]:
def getScrollers():
    #Getting the tags of the  scrollers for time periods
    loaded=False
    while loaded == False:
        try:
            scroll_tags = driver.find_elements_by_tag_name('g') #The scrollers are over the tag 'g'
            scrollers=[]

            for i in range(len(scroll_tags)):
                role = scroll_tags[i].get_attribute("role") # Scrollers has the exlusive attribute 'role'
                if role != None:
                    #print(scrollers[i].get_attribute("role"), scrollers[i].get_attribute("transform"))
                    scrollers.append(scroll_tags[i])
            loaded=True
            if len(scrollers) == 2:
                print('Scroller elements load successfully')
            else:
                print('Something went wrong with the scrollers')
        except:
            print('Loading scrollers...')
    return scrollers

In [12]:
def scrap(zeroi=-17, zerof=38, max_tries=3,offseti=1 ,offsetf=1,driver=0, wait_loading_time=0,verbose_rs=False, database_rs=[]):
    
    sleep(wait_loading_time)
    #Checking where are the scrollers
    si=scrollers[0].get_attribute("transform")
    sf=scrollers[1].get_attribute("transform")

    xi,yi,xf,yf=getScrollerPosition(si,sf)

    step=zerof-zeroi

    it=0
    tries=0
    loaded= False
    
    # Movig the scrollers for time periods
    
    try:
        while loaded == False:
            print('Fitting the area to scan', end="")
            while (xi != zeroi+step*it or xf != zerof+step*it) and tries < max_tries:
                action=ActionChains(driver)
                action.drag_and_drop_by_offset(scrollers[0],zeroi+step*it-xi-offseti,0)
                action.drag_and_drop_by_offset(scrollers[1],zerof+step*it-xf+offsetf,0)
                action.perform()

                si=scrollers[0].get_attribute("transform")
                sf=scrollers[1].get_attribute("transform")
                xi,yi,xf,yf=getScrollerPosition(si,sf)
                sleep(1)
                tries+=1
                print('.',end="")
            
            print("")
            sleep(1)
            last_day=readscreen(database = database_rs, verbose=verbose_rs) ## Getting info function
            it+=1
            tries=0
            print('Iteracion: ',it)
            if last_day == date.today():
                print('Space completed in ',it,'iterations')
                loaded=True
    except:
        print('Space completed in ',it,'iterations [except]')
    print('Scrap function completed')
    

In [13]:
def save(name,database=[]):
    df=pd.DataFrame(database)
    df=df.drop_duplicates()
    df=df.reset_index()
    df.rename(columns={0:'date',1:'open',2:'low',3:'high',4:'close',5:'volume'},inplace=True)
    df=df.drop(columns='index')
    df.to_csv('./'+name+'-mxn.csv',index=False)

In [14]:
def loadWebpage(URL=''):
    print('Opening browser...')
    driver = webdriver.Firefox()
    driver.get(URL)
    print('Maximizing window...')
    driver.maximize_window()
    print('Scrolling into graph...')
    driver.execute_script("window.scrollTo(0, 200)") 
    return driver

## Scrapper

In [16]:
for i in coins:
    database=[]
    driver = loadWebpage(URLs[i])
    scrollers = getScrollers()
    scrap(driver=driver,database_rs=database)
    save(i,database)
    print(i, ' Scrapped succesfully')
    driver.close()

Opening browser...
Maximizing window...
Scrolling into graph...
Scroller elements load successfully
Fitting the area to scan...

 103 Candels detected



Iteracion:  1
Fitting the area to scan...

 48 Candels detected



Iteracion:  2
Space completed in  2 iterations
Scrap function completed
btc  Scrapped succesfully
Opening browser...
Maximizing window...
Scrolling into graph...
Scroller elements load successfully
Fitting the area to scan...

 70 Candels detected



Iteracion:  1
Fitting the area to scan...

 33 Candels detected



Iteracion:  2
Space completed in  2 iterations
Scrap function completed
eth  Scrapped succesfully
Opening browser...
Maximizing window...
Scrolling into graph...
Scroller elements load successfully
Fitting the area to scan...

 53 Candels detected



Iteracion:  1
Fitting the area to scan...

 24 Candels detected



Iteracion:  2
Space completed in  2 iterations
Scrap function completed
xrp  Scrapped succesfully
Opening browser...
Maximizing window...
Scrolling into graph...
Scroller elements load successfully
Fitting the area to scan...
Space completed in  0 iterations [except]
Scrap function completed
ltc  Scrapped succesfully


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59270): Max retries exceeded with url: /session/8617ab1b-c40f-9a43-8702-afc71d8e4103/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x118bcb9d0>: Failed to establish a new connection: [Errno 61] Connection refused'))

## Other Sources

In [ ]:
import requests
from datetime import date

In [ ]:
books={'Bitcoin':'btc_mxn',
       'Ethereum':'eth_mxn',
       'Ripple':'xrp_mxn',
       'LTC':'ltc_mxn',
       'Bitcoin Cash':'bch_mxn',
       'True Dollar':'tusd_mxn',
       'Mana':'mana_mxn',
       'Golem':'gnt_mxn',
       'BAT':'bat_mxn',
       'DAI':'dai_mxn'}

coins=['bitcoin','ethereum','xrp','litecoin','bitcoin-cash','trueusd','decentraland','golem-network-tokens',
       'basic-attention-token','multi-collateral-dai']
start='20120428'

end= date.today()
end= end.strftime("%Y%m%d")

url=[]
for coin in coins:
    url.append('https://coinmarketcap.com/currencies/'+coin+'/historical-data/?start='+start+'&end='+end)

In [ ]:
url[0]